<a href="https://colab.research.google.com/github/YashK07/Ensembling-Machine-Learning-/blob/main/Stacking_Ensembling_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import xgboost
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
# Read the dataset
data = pd.read_csv("collegePlace.csv")

data = pd.get_dummies(data, columns=['Gender', 'Stream'])
data.head()

,Age,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot,Gender_Female,Gender_Male,Stream_Civil,Stream_Computer Science,Stream_Electrical,Stream_Electronics And Communication,Stream_Information Technology,Stream_Mechanical
0,22,1,8,1,1,1,0,1,0,0,0,1,0,0
1,21,0,7,1,1,1,1,0,0,1,0,0,0,0
2,22,1,6,0,0,1,1,0,0,0,0,0,1,0
3,21,0,8,0,1,1,0,1,0,0,0,0,1,0
4,22,0,8,1,0,1,0,1,0,0,0,0,0,1


In [ ]:
# @title Internships

from matplotlib import pyplot as plt
data['Internships'].plot(kind='hist', bins=20, title='Internships')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
# Split data into features and target
X = data.drop(columns=['PlacedOrNot'])
y = data['PlacedOrNot']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [4]:
from sklearn.preprocessing import StandardScaler
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


No null values.

The data looks balanced, so we will choose accuracy as our metric.

Here,
*   1 - Benign
*   0 - Malignant




Takeaways :


1. The data doesnot require any preprocessing.
2. EDA is not as such required so we move ahead to the modeling part.




**K fold cross Validation**

In [16]:
import xgboost
import lightgbm
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Assuming `X` is your feature matrix and `y` is your target variable


# Create instances of the classifiers
catboost = CatBoostClassifier()
dtc = DecisionTreeClassifier(max_depth=5, min_samples_split=5, min_samples_leaf=2)
rfc = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=5, min_samples_leaf=2)
knn = KNeighborsClassifier(n_neighbors=5)
xgb = xgboost.XGBClassifier(
    max_depth=3, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8
)
# Create the stacking classifier with the preprocessor
stacking_classifier = StackingClassifier(
    estimators=[
        ('dtc', dtc),
        ('rfc', rfc),
        ('xgb', xgb),
        ('knn', knn),
        ('catboost', catboost)
    ],
    final_estimator=LogisticRegression(),
)

# Perform cross-validation
score = cross_val_score(stacking_classifier, X, y, cv=5, scoring='accuracy', n_jobs=-1)

# Print the accuracy score
print("The accuracy score of StackingClassifier is:", score.mean())


The accuracy score of StackingClassifier is: 0.8732433951658235


In [ ]:
clf = [dtc,rfc,xgb,knn,catboost]
for algo in clf:
    score = cross_val_score( algo,X,y,cv = 5,scoring = 'accuracy')
    print("The accuracy score of {} is:".format(algo),score.mean())


# Stacking

In [11]:
# dtc =  DecisionTreeClassifier()
# rfc = RandomForestClassifier()
# knn =  KNeighborsClassifier()
# xgb = xgboost.XGBClassifier()
dtc = DecisionTreeClassifier(max_depth=5, min_samples_split=5, min_samples_leaf=2)
rfc = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=5, min_samples_leaf=2)
knn = KNeighborsClassifier(n_neighbors=5)
xgb = xgboost.XGBClassifier(
    max_depth=3, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8
)
clf = [('dtc',dtc),('rfc',rfc),('knn',knn),('xgb',xgb)] #list of (str, estimator)

**The ensemble stacking model achives the highest accuracy than any other model

*   List item
*   List item

taken alone.**

In [13]:
lr = LogisticRegression()
stack_model = StackingClassifier( estimators = clf,final_estimator = lr)
score = cross_val_score(stack_model,X,y,cv = 5,scoring = 'accuracy')
print("The accuracy score of is:",score.mean())

The accuracy score of is: 0.8702079820123665


In [17]:
individual_scores = {}
classifiers = [dtc, rfc, xgb, knn, catboost, stacking_classifier]
for classifier in classifiers:
    scores = cross_val_score(classifier, X, y, cv=5, scoring='accuracy', n_jobs=-1)
    classifier_name = classifier.__class__.__name__
    individual_scores[classifier_name] = scores.mean()

for classifier_name, score in individual_scores.items():
    print(f"The accuracy score of {classifier_name} is: {score}")

The accuracy score of DecisionTreeClassifier is: 0.8712209219797753
The accuracy score of RandomForestClassifier is: 0.8584087076498544
The accuracy score of XGBClassifier is: 0.8688577739167958
The accuracy score of KNeighborsClassifier is: 0.8260269928060822
The accuracy score of CatBoostClassifier is: 0.8769522089926811
The accuracy score of StackingClassifier is: 0.8715587578993989
